In [1]:
from pyspark.sql import SparkSession

spark = (SparkSession
 .builder
 .appName("Nombre_App") 
 .getOrCreate())

# MLlib hands-on: RandomForest para predecir la severidad del retraso

## Descripción de las variables

El dataset está compuesto por las siguientes variables:

1. **Year** 2008
2. **Month** 1
3. **DayofMonth** 1-31
4. **DayOfWeek** 1 (Monday) - 7 (Sunday)
5. **DepTime** hora real de salida (local, hhmm)
6. **CRSDepTime** hora prevista de salida (local, hhmm)
7. **ArrTime** hora real de llegada (local, hhmm)
8. **CRSArrTime** hora prevista de llegada (local, hhmm)
9. **UniqueCarrier** código del aparato
10. **FlightNum** número de vuelo
11. **TailNum** identificador de cola: aircraft registration, unique aircraft identifier
12. **ActualElapsedTime** tiempo real invertido en el vuelo
13. **CRSElapsedTime** en minutos
14. **AirTime** en minutos
15. **ArrDelay** retraso a la llegada, en minutos: se considera que un vuelo ha llegado "on time" si aterrizó menos de 15 minutos más tarde de la hora prevista en el Computerized Reservations Systems (CRS).
16. **DepDelay** retraso a la salida, en minutos
17. **Origin** código IATA del aeropuerto de origen
18. **Dest** código IATA del aeropuerto de destino
19. **Distance** en millas
20. **TaxiIn** taxi in time, in minutes
21. **TaxiOut** taxi out time in minutes
22. **Cancelled** *si el vuelo fue cancelado (1 = sí, 0 = no)
23. **CancellationCode** razón de cancelación (A = aparato, B = tiempo atmosférico, C = NAS, D = seguridad)
24. **Diverted** *si el vuelo ha sido desviado (1 = sí, 0 = no)
25. **CarrierDelay** en minutos: El retraso del transportista está bajo el control del transportista aéreo. Ejemplos de sucesos que pueden determinar el retraso del transportista son: limpieza de la aeronave, daño de la aeronave, espera de la llegada de los pasajeros o la tripulación de conexión, equipaje, impacto de un pájaro, carga de equipaje, servicio de comidas, computadora, equipo del transportista, problemas legales de la tripulación (descanso del piloto o acompañante) , daños por mercancías peligrosas, inspección de ingeniería, abastecimiento de combustible, pasajeros discapacitados, tripulación retrasada, servicio de inodoros, mantenimiento, ventas excesivas, servicio de agua potable, denegación de viaje a pasajeros en mal estado, proceso de embarque muy lento, equipaje de mano no válido, retrasos de peso y equilibrio.
26. **WeatherDelay** en minutos: causado por condiciones atmosféricas extremas o peligrosas, previstas o que se han manifestado antes del despegue, durante el viaje, o a la llegada.
27. **NASDelay** en minutos: retraso causado por el National Airspace System (NAS) por motivos como condiciones meteorológicas (perjudiciales pero no extremas), operaciones del aeropuerto, mucho tráfico aéreo, problemas con los controladores aéreos, etc.
28. **SecurityDelay** en minutos: causado por la evacuación de una terminal, re-embarque de un avión debido a brechas en la seguridad, fallos en dispositivos del control de seguridad, colas demasiado largas en el control de seguridad, etc.
29. **LateAircraftDelay** en minutos: debido al propio retraso del avión al llegar, problemas para conseguir aterrizar en un aeropuerto a una hora más tardía de la que estaba prevista.

**IMPORTANTE:** Vamos a emplear una versión reducida del dataset de vuelos para entender la forma de trabajar con MLlib, y en particular, los mecanimos de exploración de combinaciones de híper-parámetros para encontrar el mejor modelo, los cuales requieren ajustar varios modelos distintos varias veces. Este proceso sería demasiado largo si utilizásemos el dataset original de 300 MB. 

<p>En la siguiente celda, descargaremos este dataset reducido de Internet mediante la ejecución del comandos de Linux `wget` y lo subiremos a HDFS  con el comando de HDFS `copyFromLocal`. Los datos guardados en HDFS son volátiles y desaparecerán cuando el cluster sea desmantelado.</p>

### Vamos a utilizar una versión reducida del dataset original para poder ver en funcionamiento el ajuste de hiper-parámetros, que requiere ajustar varios modelos

#### Descargamos el CSV reducido y lo subimos a HDFS (esta vez no es Google Cloud Storage)

In [2]:
!wget https://raw.githubusercontent.com/olbapjose/xapi-clojure/master/flights_jan08.csv
#!hdfs dfs -copyFromLocal flights_jan08.csv /tmp

--2021-10-04 05:53:18--  https://raw.githubusercontent.com/olbapjose/xapi-clojure/master/flights_jan08.csv
Resolviendo raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Conectando con raw.githubusercontent.com (raw.githubusercontent.com)[185.199.109.133]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 9719583 (9,3M) [text/plain]
Guardando como: “flights_jan08.csv”

flights_jan08.csv   100%[===================>]   9,27M  18,2MB/s    en 0,5s    

2021-10-04 05:53:19 (18,2 MB/s) - “flights_jan08.csv” guardado [9719583/9719583]



#### Ahora lo leemos desde la ruta /tmp/flights_jan08.csv de HDFS

In [4]:
# Leemos el dataset de HDFS. Esta operación todavía no hace la lectura hace una pasada 
# sobre los datos para inferir el esquema
flightsDF = spark.read.option("header", "true")\
                      .option("inferSchema", "true")\
                      .csv("Datasets/flights_jan08.csv")

In [7]:
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import *
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline

In [5]:
flightsDF.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- Ca

Vamos a utilizar algunos transformadores habituales, y un algoritmo RandomForest que es un estimador. Utilizaremos:

* StringIndexer para convertir variables tipo String en variables categóricas pero cuyos valores son números reales con la parte decimal a 0, tal como necesitan los algoritmos de Spark.
* Bucketizer para discretizar la columna de ArrDelay sin dar nombre a las categorías, solo numeros. Será nuestra variable target.
* VectorAssembler para unir las columnas de las features en una sola de tipo vector
* RandomForest que es un estimador, como algoritmo de predicción de la severidad
* Pipeline, que es un estimador y que incluirá todos los elementos anteriores.

In [8]:
cleanFlightsDF = flightsDF.where("ArrDelay != 'NA' and DepDelay != 'NA' and DepTime != 'NA' and ArrTime != 'NA'")\
                          .withColumn("ArrDelay", F.col("ArrDelay").cast(IntegerType()))\
                          .withColumn("DepDelay", F.col("DepDelay").cast(IntegerType()))\
                          .withColumn("ArrTime", F.col("ArrTime").cast(IntegerType()))\
                          .withColumn("DepTime", F.col("DepTime").cast(IntegerType()))

In [9]:
# Definimos tres categorías: <15, entre 15 y 60, >60. 
# Cada una se codifica con un número real: 0.0, 1.0, 2.0 

splitsDelays = [-float("inf"), 15, 60, float("inf")]
arrDelayBucketizer = Bucketizer(splits=splitsDelays, 
                                inputCol="ArrDelay", 
                                outputCol="ArrDelayBucketed")

In [10]:
# Definimos varias franjas: 00:00 - 06:00, 06:00 - 12:00, 12:00 - 18:00, 
#                           18:00 - 22:00, 22:00 - 00:00

splitsDepTime = [-1, 600, 1200, 1800, 2200, 2500]
depTimeBucketizer = Bucketizer(splits=splitsDepTime, 
                               inputCol="DepTime", 
                               outputCol="DepTimeBucketed")

In [11]:
# Dividimos en train y test de manera aleatoria usando la semilla 123 para los números aleatorios, para que sea reproducible. 
# Esto devuelve una lista de dos DataFrames. La división hará que el primer elemento de la lista sea un DF 
# con aprox. el 70 % de las filas. Lo usaremos para entrenar. El otro DF tendrá aprox el 30 % restante de las filas. 

splits = cleanFlightsDF.randomSplit([0.7, 0.3], 
                                    seed = 123)
trainDF = splits[0].cache() # El primer DF tiene el 70 % de los datos
testDF = splits[1].cache()

In [ ]:
# Si quisiera generar la lista con todos los StringIndexer para TODAS las columnas:
# categoricas = ["asdñfkj", "añdslkjadf"]
# indexerList = [StringIndexer(inputCol=c, 
#                              outputCol=c + "indexed", 
#                              handleInvalid="skip") for c in categoricas]

In [12]:
# Indexamos las columnas categóricas Origin, Dest y DayOfWeek, para traducirlas a reales con la parte decimal a 0.
# Recordemos que esto también introduce metadatos adicionales en la columna resultante para indicar que, aunque sea
# una columna de números reales, en realidad están representando categorías y debe ser tratada como tal por el algoritmo
originIndexer = StringIndexer(inputCol = "Origin", 
                              outputCol="OriginIndexed", 
                              handleInvalid="skip")
destIndexer = StringIndexer(inputCol = "Dest", 
                            outputCol="DestIndexed", 
                            handleInvalid="skip")
dowIndexer = StringIndexer(inputCol = "DayOfWeek", 
                           outputCol="DayOfWeekIndexed", 
                           handleInvalid="skip")

In [13]:
# Ahora unimos todas las columnas que se usarán como variables en una sola columna de tipo vector llamada featuresVector
vectorAssembler = VectorAssembler(inputCols = ["DepDelay", 
                                               "DepTimeBucketed", 
                                               "OriginIndexed", 
                                               "DestIndexed", 
                                               "DayOfWeekIndexed"], 
                                  outputCol = "featuresVector")

In [14]:
randomForest = RandomForestClassifier(featuresCol = "featuresVector",
                                      labelCol = "ArrDelayBucketed",
                                      numTrees = 50, 
                                      maxBins=100)

pipeline = Pipeline(stages=[arrDelayBucketizer, 
                            depTimeBucketizer, 
                            dowIndexer, 
                            originIndexer, 
                            destIndexer, 
                            vectorAssembler, 
                            randomForest])

Aplicamos fit para ajustar el pipeline completo. Lo que esto hace es, para cada etapa:
- Si la etapa es un Transformer, invoca al método transform() pasándole el DF que recibe de la etapa anterior,
  y envía el resultado (DF transformado) a la etapa siguiente del pipeline.
- Si la etapa es un Estimator, invoca al método fit() pasándole como argumento el DF recibido de la etapa anterior,
  y después invoca transform() sobre el objeto devuelto por fit, pasando de nuevo dicho DF como argumento. 
  Finalmente el DF devuelvo por transform es enviado a la etapa siguiente del pipeline.

In [15]:
pipelineModel = pipeline.fit(trainDF)

In [22]:
pipelineModel.stages

[Bucketizer_ff3db6b6da04,
 Bucketizer_6ba0b07155b2,
 StringIndexerModel: uid=StringIndexer_af985d2375f2, handleInvalid=skip,
 StringIndexerModel: uid=StringIndexer_ba8d3b35fa93, handleInvalid=skip,
 StringIndexerModel: uid=StringIndexer_e7fcce327d46, handleInvalid=skip,
 VectorAssembler_29e2603e6c5d,
 RandomForestClassificationModel: uid=RandomForestClassifier_a2d064398510, numTrees=50, numClasses=3, numFeatures=5]

In [23]:
pipelineModel.stages[0].getSplits()

[-inf, 15.0, 60.0, inf]

Vamos a hacer predicciones sobre los datos de test. Esto devolverá un nuevo DF al que se le han añadido varias columnas al final:
1. **`rawPrediction`**: magnitud que tiene una interpretación diferente según el algoritmo pero que intuitivamente nos da una
  medida de la confianza en cada posible clase (cuanto mayor, más confianza se tiene en que esa es la clase del ejemplo). En
  nuestro caso será un vector de 3 números reales puesto que nuestro problema tiene 3 clases
2. **`probability`**: vector de probabilidades de que el ejemplo pertenezca a cada una de las 3 clases de nuestro problema
3. **`prediction`**: clase para la cual la rawProbability es mayor.

In [16]:
predictionsDF = pipelineModel.transform(testDF)

In [17]:
print("Hay {0} ejemplos de entrenamiento".format(trainDF.count()))

Hay 69152 ejemplos de entrenamiento


In [19]:
predictionsDF.toPandas()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,LateAircraftDelay,ArrDelayBucketed,DepTimeBucketed,DayOfWeekIndexed,OriginIndexed,DestIndexed,featuresVector,rawPrediction,probability,prediction
0,2008,1,3,4,600,600,711,715,WN,2111,...,NA,0.0,1.0,0.0,4.0,7.0,"[0.0, 1.0, 4.0, 7.0, 0.0]","[47.6674061185668, 2.2904215096657694, 0.04217...","[0.9533481223713361, 0.045808430193315396, 0.0...",0.0
1,2008,1,3,4,601,600,719,720,WN,1940,...,NA,0.0,1.0,0.0,27.0,26.0,"[1.0, 1.0, 27.0, 26.0, 0.0]","[47.77524112449889, 2.1812078517622133, 0.0435...","[0.955504822489978, 0.043624157035244276, 0.00...",0.0
2,2008,1,3,4,602,600,733,730,WN,3233,...,NA,0.0,1.0,0.0,0.0,4.0,"[2.0, 1.0, 0.0, 4.0, 0.0]","[47.4190118106295, 2.5387376472074616, 0.04225...","[0.9483802362125902, 0.05077475294414924, 0.00...",0.0
3,2008,1,3,4,605,605,723,730,WN,3330,...,NA,0.0,1.0,0.0,4.0,30.0,"[0.0, 1.0, 4.0, 30.0, 0.0]","[47.739289505198556, 2.210250050316093, 0.0504...","[0.9547857901039712, 0.04420500100632187, 0.00...",0.0
4,2008,1,3,4,607,600,831,810,WN,922,...,0,1.0,1.0,0.0,11.0,3.0,"[7.0, 1.0, 11.0, 3.0, 0.0]","[47.50520246680057, 2.39803243352218, 0.096765...","[0.9501040493360116, 0.04796064867044361, 0.00...",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29533,2008,1,31,4,2342,2005,47,2120,WN,470,...,207,2.0,4.0,0.0,23.0,16.0,"[217.0, 4.0, 23.0, 16.0, 0.0]","[0.052585987261146494, 2.305320672305464, 47.6...","[0.0010517197452229302, 0.04610641344610929, 0...",2.0
29534,2008,1,31,4,2344,1940,320,2205,WN,1777,...,223,2.0,4.0,0.0,1.0,8.0,"[244.0, 4.0, 1.0, 8.0, 0.0]","[0.6813745318006782, 2.4861041575955904, 46.83...","[0.013627490636013569, 0.04972208315191183, 0....",2.0
29535,2008,1,31,4,2344,2105,116,2235,WN,1274,...,156,2.0,4.0,0.0,0.0,40.0,"[159.0, 4.0, 0.0, 40.0, 0.0]","[0.5843394913694113, 2.0243409239836883, 47.39...","[0.011686789827388232, 0.04048681847967378, 0....",2.0
29536,2008,1,31,4,2352,2045,256,2330,WN,1442,...,167,2.0,4.0,0.0,1.0,2.0,"[187.0, 4.0, 1.0, 2.0, 0.0]","[0.6813745318006782, 2.3278953895994037, 46.99...","[0.013627490636013567, 0.04655790779198809, 0....",2.0


### Evaluación del modelo (evaluamos las predicciones que hemos hecho sobre el DF de test)

In [24]:
# Select (prediction, true label) and compute test error
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Objeto evaluador, para evaluar las predicciones. Compara una columna de predicciones con la columna target del verdadero valor
# Hay varias métricas posibles, pero nosotros hemos elegido la más simple: porcentaje de acierto en clasificación.
evaluator = MulticlassClassificationEvaluator(
    labelCol="ArrDelayBucketed", predictionCol="prediction", metricName="accuracy")

accuracy = evaluator.evaluate(predictionsDF)

print("Test Error = %g " % (1.0 - accuracy))

Test Error = 0.0788476 


### Ajuste de híper-parámetros utilizando Cross Validation sobre el subconjunto de train

In [25]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit

# Objeto ParamGrid al que le tenemos que indicar cada uno de los parámetros sobre los que queremos buscar,
# y la lista de valores posibles que queremos probar con cada uno. Deben ser parámetros tomados directamente
# del objeto estimador que se añadió al pipeline (no pueden ser de ningún otro modelo). En nuestro caso este objeto
# está almacenado en la variable randomForest que habíamos creado en celdas anteriores.
paramGrid = ParamGridBuilder()\
    .addGrid(randomForest.numTrees, [50, 100, 150])\
    .addGrid(randomForest.maxDepth, [3, 4, 5])\
    .build() # como tenemos 2 parámetros con 3 valores posibles cada uno, hay 9 combinaciones posibles

# CrossValidator es un estimador. Cuando invocamos a fit(), probará todas las combinaciones de valores de los 
# parámetros, invocando con cada combinación al método fit() del objeto pipeline que le hemos pasado como argumento
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)

# Como hemos hecho 3 folds, habrá que entrenar 3 veces cada modelo candidato (cada combinación de parámetros)
# y obtener su media de accuracy. En total vamos a entrenar 27 modelos

cvModel = crossval.fit(trainDF)
cvModel.bestModel

PipelineModel_34024880dbb1

In [28]:
cvModel.extractParamMap()

{Param(parent='CrossValidatorModel_ba528baa59ff', name='seed', doc='random seed.'): -7553321941270344866,
 Param(parent='CrossValidatorModel_ba528baa59ff', name='numFolds', doc='number of folds for cross validation'): 3,
 Param(parent='CrossValidatorModel_ba528baa59ff', name='foldCol', doc="Param for the column name of user specified fold number. Once this is specified, :py:class:`CrossValidator` won't do random k-fold split. Note that this column should be integer type with range [0, numFolds) and Spark will throw exception on out-of-range fold numbers."): '',
 Param(parent='CrossValidatorModel_ba528baa59ff', name='estimator', doc='estimator to be cross-validated'): Pipeline_66810f493438,
 Param(parent='CrossValidatorModel_ba528baa59ff', name='estimatorParamMaps', doc='estimator param maps'): [{Param(parent='RandomForestClassifier_a2d064398510', name='numTrees', doc='Number of trees to train (>= 1).'): 50,
   Param(parent='RandomForestClassifier_a2d064398510', name='maxDepth', doc='Ma

#### El objeto RandomForestModel (modelo ajustado presente en la última etapa del pipeline ya ajustado) dispone de ciertos atajos para recuperar valores de algunos parámetros (por ejemplo getNumTrees) pero no de otros (por ejemplo, no existe getMaxDepth)

In [26]:
rf = cvModel.bestModel.stages[6]
print("Número óptimo de árboles: {0}".format(rf.getNumTrees))
print("Max depth óptimo: {0}".format(rf.getOrDefault('maxDepth')))

Número óptimo de árboles: 50
Max depth óptimo: 5
